<a href="https://colab.research.google.com/github/SPS-L/pyeplan/blob/master/examples/5_bus_MG_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a small microgrid network.

## Introduction

In this example we consider a small microgrid network consisting of 5 buses. A bus corresponds to a location where the power is either injected into the system (e.g., a generator) or consumed (an electrical load). There is an load connected at each bus. To define the load and generation profiles, one representative day with three operational hours is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, battery units, solar units and diesel generation units. Their potential location in the network will be defined using the input system data.

We showcase how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the network.

This example is designed to be run on Google Colab. If this is the first time you use Colab, please take a 3-minute break and check out [this video](https://youtu.be/inN8seMm7UI).

To get a quickstart, you can simply click on "Runtime->Run all" and accept the prompt. This will execute the entire notebook and you can read through it like a report.

The following commands are used to download the system data and intialise the Colab platform to run PyEPLAN

In [18]:
#@title 
!rm -r sample_data 
!rm -r 5_bus 

rm: cannot remove 'sample_data': No such file or directory


In [19]:
#@markdown  

!apt-get install -qq subversion                                                &> /dev/null
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/5_bus              &> /dev/null

In [20]:
#@title 

!apt-get install -y -qq glpk-utils  &> /dev/null
!apt-get install -y -qq coinor-cbc  &> /dev/null

In [21]:
#@markdown PyEPLAN software is installed using the following command. 

!pip install -q pyeplan  &> /dev/null

In [22]:
#@title 

!pip install git+git://github.com/SPS-L/pyeplan@master  &> /dev/null

## Using the invesment and operation planning module from PyEPLAN 

In [23]:
#@markdown This command is used to add the PyEPLAN Planning Module to Colab environment.  

from pyeplan import inosys

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [24]:
#@markdown The following command is used to intialise the Planning module of PyEPLAN

inosys = inosys('5_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3,)

## Defining the system data

Input files containing the system data are added to PyEPLAN using csv files. These files contain the load demand for each planning hour and load point, candidate and existing generators, network layout data and load and renewable generation profiles. 

The data input and format of different csv files accepted by PyEPLAN software is defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).  

As a basis, the user should to define the load demand and at least one generation input to be able to utilise the investment and operation modules of PyEPLAN.

Some of the data inputs include:

### Electricity load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [25]:
#@title 

import pandas as pd
pd.read_csv("5_bus/prep_dist.csv")

,0
0,24.6
1,55.4
2,80.7


### Potential investment candidates

#### Battery Units

In [26]:
#@title 

pd.read_csv("5_bus/cbat_dist.csv")

,bus,icost,ocost,scost,ec,ed,emax,emin,eini,pmax,pmin,qmax,qmin
0,0,347.2176,0,0,0.92,0.92,1200,0,600,100,0,0,0
1,4,347.2176,0,0,0.92,0.92,1200,0,600,100,0,0,0


In [27]:

#@markdown Change the battery system capital investment cost

# reading the csv file
df = pd.read_csv("5_bus/cbat_dist.csv")
old_bcost = df.loc[0, 'icost'] 
#bat_inv_cost = "247.2176" #@param [147.2176, 247.2176, 347.2176]
bat_inv_cost= 347.2176 #@param {type:"slider", min:150, max:400, step:1}

# updating the column value/data
df['icost'] = df['icost'].replace({old_bcost: bat_inv_cost})
  
# writing into the file
df.to_csv("5_bus/cbat_dist.csv", index=False)

#pd.read_csv("5_bus/cbat_dist.csv")

#### Solar PV Units

In [28]:
#@title 

pd.read_csv("5_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,108.766,0,0,0,50,-50,50
1,4,108.766,0,0,0,50,-50,50


In [29]:
#@markdown Change the solar unit capital investment cost

# reading the csv file
df = pd.read_csv("5_bus/csol_dist.csv")
old_pvcost = df.loc[0, 'icost'] 
pv_inv_cost =108  #@param {type:"slider", min:80, max:130, step:1}
#pv_inv_cost = "118.766" #@param [88.766	, 108.766	, 118.766	]

# updating the column value/data
df['icost'] = df['icost'].replace({old_pvcost: pv_inv_cost})
  
# writing into the file
df.to_csv("5_bus/csol_dist.csv", index=False)



#### Diesel/Fossil Units

In [30]:
#@title 

pd.read_csv("5_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12.0345,0.27,0,0,100,0,0,3


In [31]:
#@title  { run: "auto" }
#@markdown Change the diesel generator capital investment cost

# reading the csv file
df = pd.read_csv("5_bus/cgen_dist.csv")
old_gcost = df.loc[0, 'icost'] 
#gen_inv_cost = "32.0345" #@param [12.0345	, 22.0345	, 32.0345	]
gen_inv_cost = 32.0345 #@param {type:"slider", min:10, max:40, step:1}
# updating the column value/data
df['icost'] = df['icost'].replace({old_gcost: gen_inv_cost})
  
# writing into the file
df.to_csv("5_bus/cgen_dist.csv", index=False)



In [32]:
#@markdown Change the diesel generator operation cost { run: "auto" }

# reading the csv file
df = pd.read_csv("5_bus/cgen_dist.csv")
old_gocost = df.loc[0, 'ocost'] 
#gen_op_cost = "0.27" #@param [0.17	, 0.27	, 0.37]
gen_op_cost = 0.27 #@param {type:"slider", min:0.10, max:0.50, step:0.01}

# updating the column value/data
df['ocost'] = df['ocost'].replace({old_gocost: gen_op_cost})
  
# writing into the file
df.to_csv("5_bus/cgen_dist.csv", index=False)



## Solving the optimization problem

PyEPLAN solves the investment and operation planning problems simultaneously. 

The optimisation problem will be solved based on the input arguments set by the user. Some of the planning options that can be set by the user include the following:

* If the user has pre-defined sizes of the investment candidates the input '**invest = True**' i.e., unit capacity = maximum capacity. Otherwise, if the user has no knowledge of the size of the candidates, these can be optimally derived by the software by setting '**invest = False**' where the unit capacity <= maximum capacity.

* The user can set solver program by defining the input to the '**solver**'. Options include both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. 

* If the user desires only to solve an operation planning problem, this can be ensured by setting input 'onlyopr = True'. 


In [33]:
#@markdown The command below is used to set the input arguments and solve the microgrid planning problem.

inosys.solve(solver = 'glpk', onlyopr = False, invest = False, )

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpb14shcx7.glpk.raw --wglp /tmp/tmpebfb44k4.glpk.glp --cpxlp
 /tmp/tmpuprohb5k.pyomo.lp
Reading problem data from '/tmp/tmpuprohb5k.pyomo.lp'...
218 rows, 133 columns, 502 non-zeros
1297 lines were read
Writing problem data to '/tmp/tmpebfb44k4.glpk.glp'...
1087 lines were written
GLPK Simplex Optimizer, v4.65
218 rows, 133 columns, 502 non-zeros
Preprocessing...
99 rows, 74 columns, 289 non-zeros
Scaling...
 A: min|aij| =  1.852e-08  max|aij| =  6.000e+02  ratio =  3.241e+10
GM: min|aij| =  5.275e-01  max|aij| =  1.896e+00  ratio =  3.594e+00
EQ: min|aij| =  2.887e-01  max|aij| =  1.000e+00  ratio =  3.463e+00
Constructing initial basis...
Size of triangular part is 99
      0: obj =   5.865550000e+06 inf =   9.377e+05 (21)
     34: obj =   5.865550000e+06 inf =   3.638e-12 (0)
*    67: obj =   1.405547920e+04 inf =   1.952e-12 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total investment and operational costs

In [34]:
#@markdown The investment and operational costs are obtained using this command

inosys.resCost()

,total costs,14055.479202
0,total investment costs,11125.319656
1,total operation costs,2930.159546


### Number and capacity of battery units installed

In [35]:
#@markdown The capacity and location of battery units installed is obtained using this command.

inosys.resBat()

,Installed Capacity (kW),Bus
Unit,,
1,0.0,0
2,0.0,4


### Number and capacity of solar units installed

In [36]:
#@markdown The capacity and location of solar units installed is obtained using this command.

inosys.resSolar()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,50.0,4


### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [37]:
#@markdown The command to wiew the capacity and location of diesel units installed is as follows.

inosys.resConv()

,Installed Capacity (kW),Bus
Unit,,
1,21.0,0


### Amount of load curtailed

In [38]:
#@markdown This command shows the amount of load in kW not served or curtailed at each node.

inosys.resCurt()

,0,1,2,3,4
Hour,,,,,
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0


## Take your learning further

This example showcases the planning modules in PyEPLAN. Different solutions may be obtained by varrying the different parameters in the system data.

As a take home exercise, use the sliders to vary the investments and operation costs of the candidate generators.

After every change you make, click on "Runtime->Run all" and accept the prompt. This will execute the entire notebook with the new values.

**How does the optimal microgrid planning solution change with variations in the different costs?**
